# The subprocess Module

## What Will be Covered?

<OL>
<LI> Introduction (http://sharats.me/the-ever-useful-and-neat-subprocess-module.html)
<LI> Simple Usage
<LI> Popen Class
<LI> Running via Shell
<LI> Getting the Return Code
<LI> IO Streams
<LI> Replacing the Shell Pipeline
<LI> Passing Environment Variables
<LI> Killing Processes
<LI> Examples
    <UL>
    <LI> Interacting with another command
    <LI> Ping command and display back its output
    </UL>
</OL>

## Introduction

<UL>
<LI> Allow to spawn new processes
<LI> Connect to the processes input/output/error pipes
<LI> Obtain the processes return codes
<LI> Good substitute for the modules <b>os</b> (system, popen, spawn), <b>popens</b>, and <b>commands</b>
</UL>

## Simple Usage

In [ ]:
import subprocess

In [ ]:
ls_output = subprocess.check_output(['ls'])

In [ ]:
subprocess.check_output(['ls', '-l'])

* The first item in the list is the executable and the rest are its command line arguments. 
* The following won't work because | (pipe) is not an argument of ls.

In [ ]:
subprocess.check_output(['ls', '|', 'wc', '-l'])

## Popen Class

<UL>
<LI> Main subprocess class
<LI> Internally used by the call, check_output and check_call classes.
</UL>

In [ ]:
# Syntax
class subprocess.Popen(args, bufsize=0, executable=None, 
                       stdin=None, stdout=None, stderr=None, 
                       preexec_fn=None, close_fds=False, shell=False,
                       cwd=None, env=None, universal_newlines=False, 
                       startupinfo=None, creationflags=0)

## Running via the Shell

In [ ]:
subprocess.call('echo $HOME', shell=True)

In [ ]:
subprocess.call('ls | wc -l', shell=True)

* If we do not set shell=True, it will be assumed that `ls` is an executable and it will not work.

In [ ]:
subprocess.call('ls -l')

* However, if args is a list, then the first item in this list is considered as the executable and the rest of the items in the list are passed as command line arguments to the program.

In [ ]:
subprocess.call(['ls', '-l'])

In [ ]:
subprocess.call('ls -l', shell=True)

In [ ]:
subprocess.call(['ls', '-l'], shell=True)

A more realistic example will look like:

In [ ]:
import sys
mycmd = 'ls'
myarg = ' -lrt'
try:
    retcode = subprocess.call(mycmd + myarg, shell=True)
    if retcode < 0:
        print(sys.stderr, "Child was terminated by signal", -retcode)
    else:
        print(sys.stderr, "Child returned", retcode)
except OSError as e:
    print(sys.stderr, "Execution failed:", e)

## Getting the Return Code

* The moment the `Popen` class is instantiated, the command starts running. 
* You can wait for it and after its done, access the return code via the returncode attribute.

In [ ]:
proc = subprocess.Popen('ls')
proc.wait()
print(proc.returncode)

## IO Streams

The simplest way to get the output of a command is to use the `check_output` function:

In [ ]:
output = subprocess.check_output('ls')
print(output)

To obtain the standard `error/output`, use:

In [ ]:
p = subprocess.Popen('ls', shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT)
(output, error) = p.communicate()
print('\nwrite:')
print("\toutput: ", output)
print("\terror:  ", error)

## Replacing Shell Pipeline

In [ ]:
output = "ls | wc -l"

In [ ]:
first_proc = subprocess.Popen(['ls'], stdout=subprocess.PIPE)
sec_proc   = subprocess.Popen(['wc', '-l'], stdin=first_proc.stdout, 
                      stdout=subprocess.PIPE)
# Allow first_proc to receive a SIGPIPE if sec_proc exits.
first_proc.stdout.close()  
output = sec_proc.communicate()[0]
print(output)

In [ ]:
output = subprocess.check_output("ls | wc -l", shell=True)
print(output)

Show the names and login times of the currently logged in users

In [ ]:
# who | cut -c 1-16,26-38 
first_proc = subprocess.Popen(['who'], stdout=subprocess.PIPE)
sec_proc = subprocess.Popen(['cut', '-c', '1-16,26-38'], 
                      stdin=first_proc.stdout, stdout=subprocess.PIPE)
# Allow first_proc to receive a SIGPIPE if sec_proc exits.
first_proc.stdout.close()  
output = sec_proc.communicate()[0]
print(output)

## Passing Environment Variables

* The env argument to Popen lets you customize the environment of the command being run. 
* You can add your own environment settings to existing ones:

In [ ]:
new_env = os.environ.copy()
new_env['MEGAVARIABLE'] = 'MEGAVALUE'
p = Popen('command', env=new_env)

## Killing Processes

In [ ]:
proc.terminate()

In [ ]:
proc.kill()

## Examples

### Interacting with Another Command

In [ ]:
run interaction.py

In [ ]:
load repeater.py

In [ ]:
load interaction.py

### Signaling Between Processes

* The `os` examples include a demonstration of signaling between processes using `os.fork()` and `os.kill()`. 
* Since each `Popen` instance provides a pid attribute with the process id of the child process, it is possible to do something similar with subprocess. For example, using this script for the child process to be executed by the parent process.

In [ ]:
run signal_parent.py

In [ ]:
# %load signal_child.py
import os
import signal
import time
import sys

pid = os.getpid()
received = False

def signal_usr1(signum, frame):
    "Callback invoked when a signal is received"
    global received
    received = True
    print('CHILD %6s: Received USR1' % pid)
    sys.stdout.flush()

print('CHILD %6s: Setting up signal handler' % pid)
sys.stdout.flush()
signal.signal(signal.SIGUSR1, signal_usr1)
print('CHILD %6s: Pausing to wait for signal' % pid)
sys.stdout.flush()
time.sleep(3)

if not received:
    print('CHILD %6s: Never received signal' % pid)


In [ ]:
# %load signal_parent.py
import os
import signal
import subprocess
import time
import sys

proc = subprocess.Popen(['python', 'signal_child.py'])
print('PARENT      : Pausing before sending signal...')
sys.stdout.flush()
time.sleep(1)
print('PARENT      : Signaling child')
sys.stdout.flush()
os.kill(proc.pid, signal.SIGUSR1)


### Ping Command and Display Back its Output

In [ ]:
p = subprocess.Popen(["ping", "-c", "10", "www.cyberciti.biz"], stdout=subprocess.PIPE)
output, err = p.communicate()
print(output)

* The only problem with above code is that output, `err = p.communicate()` will block next statement till ping is completed i.e. you will not get real time output from the ping command. So you can use the following code to get real time output:

In [ ]:
cmdping = "ping -c10 www.cyberciti.biz"
p = subprocess.Popen(cmdping, shell=True, stderr=subprocess.PIPE)
while True:
    out = p.stderr.read(1)
    if out == '' and p.poll() != None:
        break
    if out != '':
        sys.stdout.write(out)
        sys.stdout.flush()